In [1]:
from decouple import config

In [2]:
config("ALPHA_VANTAGE_API_KEY")

'B9YN3L7WUIYJMRS1'

In [3]:
assert config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) is not None

In [4]:
ALPHA_VANTAGE_API_KEY = config("ALPHA_VANTAGE_API_KEY", default=None, cast=str) 

In [5]:
import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key

api_key = ALPHA_VANTAGE_API_KEY

params = {
    "api_key": ALPHA_VANTAGE_API_KEY,
    "ticker": "AAPL",
}

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval=5min&apikey={api_key}'.format(
    **params
)

r = requests.get(url)
data = r.json()

In [9]:
dataset_key = list(data.keys())[1]
dataset_key

'Time Series (5min)'

In [13]:
results = data[dataset_key]
# for timestamp_str in results.keys():
#     print(timestamp_str)

In [41]:
timestamp_str = list(results.keys())[0]

In [42]:
result_val = results.get(timestamp_str)
result_val

{'1. open': '228.0800',
 '2. high': '228.1000',
 '3. low': '228.0600',
 '4. close': '228.1000',
 '5. volume': '786'}

In [43]:
import pytz
from datetime import datetime
from decimal import Decimal

def transform_alpha_vantage_result(timestamp_str, result_val):
    # unix_timestamp = result.get('t') / 1000
    # utc_timestamp = datetime.fromtimestamp(unix_timestamp, tz=pytz.timezone('UTC'))
    timestamp_format = '%Y-%m-%d %H:%M:%S'
    eastern = pytz.timezone("US/Eastern")
    utc = pytz.utc
    timestamp = eastern.localize(datetime.strptime(timestamp_str, timestamp_format)).astimezone(utc)
    return {
     'open_price': Decimal(result_val['1. open']),
     'close_price': Decimal(result_val['4. close']),
     'high_price': Decimal(result_val['2. high']),
     'low_price': Decimal(result_val['3. low']),
     'volume': int(result_val['5. volume']),
     'time': timestamp,
    }

In [44]:
transform_alpha_vantage_result(timestamp_str, result_val)

{'open_price': Decimal('228.0800'),
 'close_price': Decimal('228.1000'),
 'high_price': Decimal('228.1000'),
 'low_price': Decimal('228.0600'),
 'volume': 786,
 'time': datetime.datetime(2024, 11, 19, 0, 55, tzinfo=<UTC>)}

In [45]:
for timestamp_str in results.keys():
    new_data = transform_alpha_vantage_result(timestamp_str, result_val)
    print(new_data)

{'open_price': Decimal('228.0800'), 'close_price': Decimal('228.1000'), 'high_price': Decimal('228.1000'), 'low_price': Decimal('228.0600'), 'volume': 786, 'time': datetime.datetime(2024, 11, 19, 0, 55, tzinfo=<UTC>)}
{'open_price': Decimal('228.0800'), 'close_price': Decimal('228.1000'), 'high_price': Decimal('228.1000'), 'low_price': Decimal('228.0600'), 'volume': 786, 'time': datetime.datetime(2024, 11, 19, 0, 50, tzinfo=<UTC>)}
{'open_price': Decimal('228.0800'), 'close_price': Decimal('228.1000'), 'high_price': Decimal('228.1000'), 'low_price': Decimal('228.0600'), 'volume': 786, 'time': datetime.datetime(2024, 11, 19, 0, 45, tzinfo=<UTC>)}
{'open_price': Decimal('228.0800'), 'close_price': Decimal('228.1000'), 'high_price': Decimal('228.1000'), 'low_price': Decimal('228.0600'), 'volume': 786, 'time': datetime.datetime(2024, 11, 19, 0, 40, tzinfo=<UTC>)}
{'open_price': Decimal('228.0800'), 'close_price': Decimal('228.1000'), 'high_price': Decimal('228.1000'), 'low_price': Decimal(